In [1]:
import os
import re
import math
import logging
import random
import functools
import hashlib

import sqlite3
import json
import pandas as pd
import numpy as np
import itertools as tt

#from collections import Counter
import matplotlib.pyplot as plt

logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %I:%M:%S')
logging.getLogger().setLevel(logging.INFO)
logging.info("Logging set.")

2023-12-05 11:01:27 INFO: Logging set.


In [2]:
# Random helper functions.
def get_integer_input(filename): 
    retval = None
    with open(filename) as IN:
        retval = [int(x) for x in IN.read().splitlines()]
    logging.info("Got {} integers from {}.".format(len(retval),filename))
    return retval

def get_string_input(filename):
    retval = None
    with open(filename) as IN:
        retval = IN.read().splitlines()
    logging.info("Got {} lines from {}.".format(len(retval),filename))
    return retval

def dot_product(vec1, vec2):
    ''' Return the dot product vec1 and vec2. '''
    assert len(vec1) == len(vec2)
    return sum([x*y for x,y in zip(vec1,vec2)])

def product(vals):
    ''' Return the product of an list of numbers. '''
    retval = 1
    assert len(vals) > 0
    for v in vals:
        retval *= v
    return retval

def get_grid_index(r, c, n, m): 
    return r*m + c

def get_grid_neighborhood(ind, n, diagonal=True):  # Used
    ''' Return the indices that are adjacent to [r,c] in a grid of size n. '''
    retval = []
    size = n*n
    aboveRow = [ind-n-1, ind-n, ind-n+1]
    belowRow = [ind+n-1, ind+n, ind+n+1]
    if diagonal == False:
        aboveRow = [ind-n]
        belowRow = [ind+n]
        
    for newInd in aboveRow:
        if 0 <= newInd and newInd < size and ind//n == (newInd//n) + 1:
            retval.append(newInd)
    for newInd in [ind-1, ind+1]:
        if 0 <= newInd and newInd < size and ind//n == newInd//n:
            retval.append(newInd)
    for newInd in belowRow:
        if 0 <= newInd and newInd < size and ind//n == (newInd//n) - 1:
            retval.append(newInd)
    return retval


# Day 1: Not Quite Lisp

## 1a. To what floor do the instructions take Santa?

## 1b. When does Santa first enter the basement?

In [3]:
filename = 'prob01input.txt'
command = get_string_input(filename)
#command =['(())','()()','(((','(()(()(','))(((((','())','))(',')))',')())())']

# Part A
for c in command:
    floor = c.count('(') - c.count(')')

# Part B
curFloor = 0
for i, c in enumerate(command[0]):
    if c == '(':
        curFloor += 1
    elif c == ')':
        curFloor -= 1
    if curFloor == -1:
        break
    
logging.info("1a.  Elevator ends on floor {}".format(floor,))
logging.info("1b.  Elevator enters basement on step {}".format(i+1,))

2023-12-05 11:01:27 INFO: Got 1 lines from prob01input.txt.
2023-12-05 11:01:27 INFO: 1a.  Elevator ends on floor 280
2023-12-05 11:01:27 INFO: 1b.  Elevator enters basement on step 1797


# Day 2:  I Was Told There Would Be No Math

## 2a.  How many total square feet of wrapping paper should they order?

## 2b.  How many total feet of ribbon should they order?


In [4]:
filename = 'prob02input.txt'
lines = get_string_input(filename)

boxes = [[int(x) for x in l.split('x')] for l in lines]

partA, partB = 0, 0
for b in boxes:   # b = [side lengths]
    surfaces = b[0] * b[1], b[0] * b[2], b[1] * b[2]  # surface areas of each face
    partA += dot_product(surfaces, [2,2,2]) + min(surfaces)
    partB += (2*(sum(b) - max(b))) + product(b)  

logging.info("2a. Total paper = {}".format(partA,))
logging.info("2b. Total ribbon = {}".format(partB,))

2023-12-05 11:01:27 INFO: Got 1000 lines from prob02input.txt.
2023-12-05 11:01:27 INFO: 2a. Total paper = 1588178
2023-12-05 11:01:27 INFO: 2b. Total ribbon = 3783758


# Day 3: Perfectly Spherical Houses in a Vacuum

## 3a. How many houses receive at least one present?

## 3b. How many houses receive at least one present if Robo-Santa helps?

In [5]:
filename = 'prob03input.txt'
dirs = get_string_input(filename)
#dirs = ['>', '^>v<', '^v^v^v^v^v']

def make_move(location, direction):
    ''' Return the location arrived at by moving one step in the plane
         indicated by direction.  
        location is a list of length two with integer indices
        direction is an 'arrow' char indicating direction to move
    '''
    assert direction in ['<', '>', '^', 'v']
    retval = location
    if direction in ['<','>']:
        retval[0] = retval[0] +1 if direction == '>' else retval[0] - 1
    else:
        retval[1] = retval[1] +1 if direction == '^' else retval[1] - 1
    return retval

# Part A
dirs = dirs[0]
santa, visitedA = [0,0], set()
visitedA.add(tuple(santa))
for d in dirs:
    santa = make_move(santa, d)
    visitedA.add(tuple(santa))

# Part B
santas, visitedB = [[0,0], [0,0]], set()
visitedB.add(tuple(santas[0]))
for i, d in enumerate(dirs):
    player = i % 2  # player 0 is Santa, player 1 is Robo-Santa
    santas[player] = make_move(santas[player], d)
    visitedB.add(tuple(santas[player]))

logging.info("3a. Houses visited: {}".format(len(visitedA),))
logging.info("3b. Houses visited with Robo-Santa: {}".format(len(visitedB),))

2023-12-05 11:01:27 INFO: Got 1 lines from prob03input.txt.
2023-12-05 11:01:27 INFO: 3a. Houses visited: 2565
2023-12-05 11:01:27 INFO: 3b. Houses visited with Robo-Santa: 2639


# Day 4:  The Ideal Stocking Stuffer

## 4a. What is the first number whose output has 5 leading zeros?

## 4b. What is the first number whose output has 6 leading zeros?

In [6]:
salts = ['abcdef','pqrstuv','bgvyzdsv']

salt, partA, partB, limit = salts[2], None, None, 10000000
for mineNumber in range(1, limit):
    coinInput = salt + str(mineNumber)
    coinHash = hashlib.md5(coinInput.encode('utf8')).hexdigest()
    if partA == None and coinHash[0:5] == '00000':
        partA = mineNumber
        logging.info("MD5({}) = {}".format(coinInput, coinHash))
    if partB == None and coinHash[0:6] == '000000':
        partB = mineNumber
        logging.info("MD5({}) = {}".format(coinInput, coinHash))
    if partA != None and partB != None:
        break

logging.info("4a: Lowest positive number giving 5 zeros: {}".format(partA,))
logging.info("4b: Lowest positive number giving 6 zeros: {}".format(partB,))

2023-12-05 11:01:27 INFO: MD5(bgvyzdsv254575) = 000004b30d481662b9cb0c105f6549b2
2023-12-05 11:01:28 INFO: MD5(bgvyzdsv1038736) = 000000b1b64bf5eb55aad89986126953
2023-12-05 11:01:28 INFO: 4a: Lowest positive number giving 5 zeros: 254575
2023-12-05 11:01:28 INFO: 4b: Lowest positive number giving 6 zeros: 1038736


# Day 5:  Doesn't He Have Intern-Elves For This?

## 5a. How many nice strings in the list?

## 5b. How many nice strings in the list?

In [7]:
filename = 'prob05input.txt'
lines = get_string_input(filename)
#lines = ['ugknbfddgicrmopn','aaa','jchzalrnumimnmhp','haegwjzuvuyypxyu','dvszwmarrgswjxmb']
#lines = ['qjhvhtzxzqqjkmpb', 'xxyxx', 'uurcxstgmygtbstg', 'ieodomkazucvgmuy']

reqA2 = re.compile(r'([a-z])\1')
reqA3 = re.compile('(ab|cd|pq|xy)')
reqB1 = re.compile(r'([a-z][a-z]).*\1')
reqB2 = re.compile(r'([a-z]).\1')
partA, partB = 0, 0
for l in lines:
    rA1 = sum([l.count(ch) for ch in ['a','e','i','o','u']])
    mA2, mA3 = reqA2.search(l), reqA3.search(l)
    mB1, mB2 = reqB1.search(l), reqB2.search(l)
    if rA1 >= 3 and mA2 != None and mA3 == None:  # Part A
        partA += 1
    if mB1 != None and mB2 != None:  # Part B
        partB += 1

logging.info("5a. Number of nice strings: {}".format(partA,))
logging.info("5b. Number of nice strings: {}".format(partB,))

2023-12-05 11:01:28 INFO: Got 1000 lines from prob05input.txt.
2023-12-05 11:01:28 INFO: 5a. Number of nice strings: 258
2023-12-05 11:01:28 INFO: 5b. Number of nice strings: 53


# Day 6:  Probably a Fire Hazard

## 6a.  After following the instructions, how many lights are lit?

## 6b.  What is the total brightness?

In [8]:
filename = 'prob06input.txt'
lines = get_string_input(filename)
#lines =['mjqjpqmgbljsphdztnvjfqwrcgsmlb','bvwbjplbgvbhsrlpgdmjqwftvncz','nppdvjthqldpwncqszvftbrmjlhg','nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg','zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw']

line_re = re.compile('^\s*((turn off)|(turn on)|(toggle))\s+(\d+),(\d+)\s+through\s+(\d+),(\d+)\s*$')

commands = []
for l in lines:
    m1 = line_re.match(l)
    if m1 == None:
        logging.info("Unable to parse {}".format(l,))
    else:
        c = [m1.groups(1)[0], 
             [int(m1.groups(1)[4]),int(m1.groups(1)[5])], 
             [int(m1.groups(1)[6]),int(m1.groups(1)[7])]]
        commands.append(c)

matA = [[0 for y in range(1000)] for x in range(1000)]
matB = [[0 for y in range(1000)] for x in range(1000)]

# run commands Part A
for c in commands:
    xmin, xmax = min(c[1][0], c[2][0]), max(c[1][0], c[2][0])
    ymin, ymax = min(c[1][1], c[2][1]), max(c[1][1], c[2][1])
    for x, y in tt.product(range(xmin, xmax+1), range(ymin, ymax+1)):
        if c[0] == 'turn on':
            matA[x][y] = 1
            matB[x][y] += 1
        elif c[0] == 'turn off':
            matA[x][y] = 0
            matB[x][y] = max(matB[x][y] - 1, 0)
        elif c[0] == 'toggle':
            matA[x][y] = (matA[x][y] + 1) % 2
            matB[x][y] += 2
        
partA = sum([sum(row) for row in matA])
partB = sum([sum(row) for row in matB])
logging.info("6a. Number of lights lit: {}".format(partA,))
logging.info("6b. Total brightness: {}".format(partB,))

2023-12-05 11:01:28 INFO: Got 300 lines from prob06input.txt.
2023-12-05 11:01:36 INFO: 6a. Number of lights lit: 569999
2023-12-05 11:01:36 INFO: 6b. Total brightness: 17836115


# Day 7:  Some Assembly Required

## 7a.  What signal is provided to wire a?

## 7b.  With updated input, what signal is provided to wire a?

In [9]:
filename = 'prob07input.txt'
lines = get_string_input(filename)
#lines = ['123 -> x','456 -> y','x AND y -> d','x OR y -> e','x LSHIFT 2 -> f','y RSHIFT 2 -> g','NOT x -> h','NOT y -> i']

# Parsing input into commands
combine_re = re.compile('^([a-z0-9]+) ((AND)|(OR)|(LSHIFT)|(RSHIFT)) ([a-z0-9]+) -> ([a-z]+)$')
assign_re = re.compile('^(NOT )?([a-z0-9]+) -> ([a-z]+)$')

def get_string_or_number(chars):
    return int(chars) if chars.isdigit() else chars

commands = []
for l in lines:
    command = []  # command = [operation, <inputs>, output wire]
    m1 = combine_re.match(l)
    m2 = assign_re.match(l)
    if m1 != None:
        op, val1, val2, wire = [m1.groups(1)[1], m1.groups(1)[0], m1.groups(1)[6], m1.groups(1)[7]]
        for chars in [op, val1, val2, wire]:
            command.append(get_string_or_number(chars))
    elif m2 != None:
        op, val1, wire = m2.groups(1)[0], m2.groups(1)[1], m2.groups(1)[2]
        op = 'ASSIGN' if op == 1 else 'NOT'
        for chars in [op, val1, wire]:
            command.append(get_string_or_number(chars))
    commands.append(command)
# Finished parsing input

initialInput = commands[:]  # make copy for part B
modulus = 2**16

def run_command(c, vals):
    """ Return to output of the command, if possible.
        c = [operation, <inputs>, output wire]
         where <inputs> is a list of wire names or integers
        If all noninteger inputs exist in vals, execute op and return val
        If not, return None.
    """
    retval = None  # assume command was executed
    # Get input values, if vals are not integers, get value from vals if set
    op, val1, val2 = c[0], c[1], c[2]
    if type(val1) != type(int()):
        val1 = vals.get(c[1], None)  # Get value if set
    if type(val2) != type(int()) and op not in ['ASSIGN','NOT']:
        val2 = vals.get(c[2], None)  # Get value if set
       
    if val1 != None and val2 != None:
        # logging.info("Executing {} with vals {} and {}".format(c, val1, val2))
        if op == 'AND':         retval = (val1 & val2) % modulus
        elif op == 'OR':        retval = (val1 | val2) % modulus
        elif op == 'LSHIFT':    retval = (val1 << val2) % modulus
        elif op == 'RSHIFT':    retval = (val1 >> val2) % modulus
        elif op == 'NOT':       retval = ((modulus-1) ^ val1) % modulus
        elif op == 'ASSIGN':    retval = (val1) % modulus
    return retval

# Part A
# Commands cannot be executed until the inputs are available.  Basic
# strategy is to attempt to execute each command.  If it cannot be
# executed, try again later.  If it can, drop it from list.  Continue
# iterating through commands until all commands have been executed.
vals = dict()
while len(commands) > 0:
    unfinished = []
    # Attempt to execute each command.
    for c in commands:
        updateWire = c[-1]
        updateVal = run_command(c, vals)
        if updateVal == None:    # Command could not be executed
            unfinished.append(c) # Save for next iteration of while loop
        else:   # Command could be executed, update vals
            vals[updateWire] = updateVal
    # Finsihed trying to execute commands.  Try again with failed commands  
    commands = unfinished
partA = vals['a']

# Part B
# Same strategy as part A but ignore command setting value for 'b'
commands = initialInput  # Reset commands
vals = {'b':partA}  # Set wire b to be output from partA
while len(commands) > 0:
    unfinished = []
    # Attempt to execute each command.
    for c in commands:
        updateWire = c[-1]
        updateVal = run_command(c, vals)
        if updateVal == None:
            unfinished.append(c)
        elif updateWire != 'b':  # Do not update wire 'b' for part b
            vals[updateWire] = updateVal
    # Finsihed trying to execute commands.  Try again with failed commands  
    commands = unfinished
partB = vals['a']

logging.info("7a. Signal on wire a: {}".format(partA,))
logging.info("7b. Signal on wire a with wire b updated: {}".format(partB,))

2023-12-05 11:01:36 INFO: Got 339 lines from prob07input.txt.
2023-12-05 11:01:36 INFO: 7a. Signal on wire a: 46065
2023-12-05 11:01:36 INFO: 7b. Signal on wire a with wire b updated: 14134


In [10]:
# After my initial program (above) I decided a better approach was to use recursion,
# so the solution in this cell uses recursion and is, imho, a cleaner approach.
# I kept both this solutions because I thought it might be of interest.

filename = 'prob07input.txt'
lines = get_string_input(filename)
#lines = ['123 -> x','456 -> y','x AND y -> d','x OR y -> e','x LSHIFT 2 -> f','y RSHIFT 2 -> g','NOT x -> h','NOT y -> i']

# Parsing input into wires
combine_re = re.compile('^([a-z0-9]+) ((AND)|(OR)|(LSHIFT)|(RSHIFT)) ([a-z0-9]+) -> ([a-z]+)$')
assign_re = re.compile('^(NOT )?([a-z0-9]+) -> ([a-z]+)$')

def get_string_or_number(chars):
    return int(chars) if chars.isdigit() else chars

wires = dict()
for l in lines:
    m1 = combine_re.match(l)
    m2 = assign_re.match(l)
    if m1 != None:
        op, val1, val2, wire = [m1.groups(1)[1], m1.groups(1)[0], m1.groups(1)[6], m1.groups(1)[7]]
        wires[wire] = [op, get_string_or_number(val1), get_string_or_number(val2)]
    elif m2 != None:
        op, val1, wire = m2.groups(1)[0], m2.groups(1)[1], m2.groups(1)[2]
        op = 'ASSIGN' if op == 1 else 'NOT'
        wires[wire] = [op, get_string_or_number(val1)]            
# Finished parsing input

backupForPartB = wires.copy()
modulus = 2**16

def execute_operation(op, val1, val2=None):
    ''' Apply op to the values and return the result.

        op: a string in ['AND','OR','LSHIFT','RSHIFT','NOT','ASSIGN']
        val1: an integer
        val2: an integer or None (if op only requires one value)
    '''
    retval = None
    assert type(val1) == type(int())
    assert (( type(val2) == type(int()) ) or ( op in ['NOT', 'ASSIGN'] ))
    if op == 'AND':         retval = (val1 & val2) % modulus
    elif op == 'OR':        retval = (val1 | val2) % modulus
    elif op == 'LSHIFT':    retval = (val1 << val2) % modulus
    elif op == 'RSHIFT':    retval = (val1 >> val2) % modulus
    elif op == 'NOT':       retval = ((modulus-1) ^ val1) % modulus
    elif op == 'ASSIGN':    retval = (val1) % modulus
    return retval

def get_value_for_wire(wire, wires):
    ''' Recursive function that determines and returns the value of wire.
         If wire is an integer or wires[wire] is an integer, just return it.
         If not, recursively compute the values needed for the operation
         and then apply the operation with those values to get the value
         of the wire.
        Once the value of the wire is computed, update that value in
         wires so it does not need to be recomputed if encountered again.
        wire: a string or integer indicating a wire name or value, respectively
        wires: dict() to store values of wires or operations indicating how to
          compute them.  wires[wireName] = wireValue
    '''
    retval = None
    if type(wire) == type(int()):
        retval = wire
    elif type(wires[wire]) == type(int()):
        retval = wires[wire]
    elif len(wires[wire]) == 3:
        op, val1, val2 = wires[wire]
        val1 = get_value_for_wire(val1, wires)
        val2 = get_value_for_wire(val2, wires)
        retval = execute_operation(op, val1, val2)
    elif len(wires[wire]) == 2:
        op, val1 = wires[wire]
        val1 = get_value_for_wire(val1, wires)
        retval = execute_operation(op, val1, None)
    wires[wire] = retval
    return retval

partA = get_value_for_wire('a', wires)
wires = backupForPartB   # Reset wires
wires['b'] = partA       # Override wire b to value from partA
partB = get_value_for_wire('a', wires)
logging.info("7a. Signal on wire a: {}".format(partA,))
logging.info("7b. Signal on wire a with wire b updated: {}".format(partB,))

2023-12-05 11:01:36 INFO: Got 339 lines from prob07input.txt.
2023-12-05 11:01:36 INFO: 7a. Signal on wire a: 46065
2023-12-05 11:01:36 INFO: 7b. Signal on wire a with wire b updated: 14134


# Day 8:  Matchsticks

## 8a.  What is the difference between the number of code characters and the number of rendered characters?

## 8b.  What is the difference between the number of encoded characters and the number of code characters?

In [11]:
filename = 'prob08input.txt'
lines = get_string_input(filename)
#lines = [r'""',r'"abc"',r'"aaa\"aaa"',r'"\x27"']

# specialChar matches \\ or \" or \x[hex][hex] , it takes four \ to match one \
specialChar = re.compile('^(.*?)((\\\\\\\\)|(\\\\")|(\\\\x[a-f0-9]{2}))(.*)$') 

def count_rendered_chars(line, specialChar):
    ''' Return the number of rendered chars in the string line.
        Basic premise is to match to the first special char (\\,\", or \ x)
        and split string, count the before string and match char, and then
        repeat until no more special chars are found.
    '''
    assert line[0] == line[-1] == '"'
    retval = 0
    remain = line[1:-1]
    m1 = specialChar.match(remain)
    while m1 != None:        
        before, remain = m1.groups(1)[0], m1.groups(1)[-1]
        assert before.count('\\') == 0  # before should contain no '\' chars
        retval += len(before) + 1  # add one for match char
        m1 = specialChar.match(remain)
    assert remain.count('\\') == 0  # remain should contain no '\' chars
    retval += len(remain)
    return retval

def count_encoded_chars(line, specialChar):
    ''' Return the number of encoded chars in the string line.
        Basic premise is to match to the first special char (\\,\", or \ x)
        and split string, count the before string and match char chars, 
        and then repeat until no more special chars are found.
    '''
    assert line[0] == line[-1] == '"'
    retval = 6  # Adding '\"' at the beginning and end of line, plus new '"'
    remain = line[1:-1]
    m1 = specialChar.match(remain)
    while m1 != None:
        before, matchChar, remain = m1.groups(1)[0], m1.groups(1)[1], m1.groups(1)[-1]
        assert before.count('\\') == 0  # before should contain no '\' chars
        retval += len(before) + len(matchChar) + 1 # add one to escape '\' char
        if matchChar == '\\\"' or matchChar == '\\\\':
            retval += 1  # add one to escape '"' or '\' char
        m1 = specialChar.match(remain)
    assert remain.count('\\') == 0  # remain should contain no '\' chars
    retval += len(remain)
    return retval

numCodeChars = sum([len(l) for l in lines])
numRenderChars = sum([count_rendered_chars(l, specialChar) for l in lines])
numEncodedChars = sum([count_encoded_chars(l, specialChar) for l in lines])
logging.info("8a. The difference: {}".format(numCodeChars - numRenderChars,))
logging.info("8b. The new difference: {}".format(numEncodedChars - numCodeChars,))

2023-12-05 11:01:36 INFO: Got 300 lines from prob08input.txt.
2023-12-05 11:01:36 INFO: 8a. The difference: 1333
2023-12-05 11:01:36 INFO: 8b. The new difference: 2046


# Day 9:  All in a Single Night?

## 9a.  What is the shortest route?

## 9b.  What is the longest route?

In [12]:
filename = 'prob09input.txt'
lines = get_string_input(filename)
    
# Parse input into graph G
G = dict()
line_re = re.compile('^(.*?) to (.*?) = (\d*)$') 
for l in lines:
    m1 = line_re.match(l)
    assert m1 != None
    u, v, dist = m1.groups(1)[0], m1.groups(1)[1], int(m1.groups(1)[2]) 
    Nu, Nv = G.get(u, dict()), G.get(v, dict())  # get neighborhoods
    Nu[v], Nv[u] = dist, dist                    # add to neighborhoods
    G[u], G[v] = Nu, Nv                          # update neighborhoods
# Finished parsing input into graph G

def extend_hamiltonian_path(dist, subpath, G, minmax='min'):
    ''' Return shortest (longest) hamiltonian path in G beginning with subpath.
        dist is an integer
        subpath is a list of vertex strings from G
        G is a 2 layer dictionary of distances; G[u][v] = distance from u to v
        minmax in ['min', 'max'] means shortest/longest path desired    
        Returns [dist, path] where dist is the distance of path and path
         begins with subpath.
    '''
    assert 0 <= len(subpath) < len(G)  # Must have subpath and be extendable
    orderFunc = min if minmax == 'min' else max
    retval = None
    u = None if len(subpath) == 0 else subpath[-1]  # get last vertex in path
    for v in [w for w in G.keys() if w not in subpath]:
        newPath = [v] if u == None else subpath.copy() + [v]
        newDist = 0 if u == None else dist + G[u][v]  # dist = 0 for 1st vertex
        shortLong = [newDist, newPath]               # default if path complete
        if len(newPath) < len(G):   # Is path unfinished?
            shortLong = extend_hamiltonian_path(newDist, newPath, G, minmax)
        if retval == None or orderFunc(retval[0],shortLong[0]) != retval[0]:
            retval = shortLong
    return retval

partA = extend_hamiltonian_path(0, [], G, 'min')
partB = extend_hamiltonian_path(0, [], G, 'max')

logging.info("9a. Shortest path has length {}".format(partA[0],))
logging.info("9b. Longest path has length {}".format(partB[0],))

2023-12-05 11:01:36 INFO: Got 28 lines from prob09input.txt.
2023-12-05 11:01:36 INFO: 9a. Shortest path has length 207
2023-12-05 11:01:36 INFO: 9b. Longest path has length 804


# Day 10:  Elves Look, Elves Say

## 10a.  What is the sequence length after 40 iterations?

## 10b.  What is the sequence length after 50 iterations?

In [13]:
seq = '3113322113'

def look_say(seq):
    ''' Return the extension of seq by applying the look-say algorithm.'''
    retval = ''
    assert len(seq) > 0
    current, currentCount = seq[0], 0
    for ch in seq[1:]:
        currentCount += 1
        if ch != current:
            # The digit current was repeated currentCount number of times 
            retval += str(currentCount) + str(current)
            # Reset for next current digit
            current, currentCount = ch, 0
    # Add last digit string of digits from seq to new sequence
    retval += str(currentCount+1) + str(current)
    return retval

partA, partB, numAsteps, numBsteps = None, None, 40, 50
for i in range(numBsteps):
    seq = look_say(seq)
    if i == numAsteps-1:
        partA = len(seq)
partB = len(seq)

logging.info("10a. Length of sequence after 40 steps: {}".format(partA,))
logging.info("10b. Length of sequence after 50 steps: {}".format(partB,))

2023-12-05 11:01:40 INFO: 10a. Length of sequence after 40 steps: 329356
2023-12-05 11:01:40 INFO: 10b. Length of sequence after 50 steps: 4666278


# Day 11:  Corporate Policy

## 11a.  What is Santa's next password?

## 11b.  What is Santa's next next password?

In [14]:
pswd = 'hxbxwxba'  # puzzle input
alphaIndex = dict(zip('abcdefghijklmnopqrstuvwxyz',range(26)))
digitIndex = dict(zip(range(26),'abcdefghijklmnopqrstuvwxyz'))

def get_digits(word):
    ''' Convert from string of letters to digit array.'''
    return [alphaIndex[ch] for ch in word]

def get_word(digits):
    ''' Convert from digit array to string of letters.'''
    return ''.join([digitIndex[d] for d in digits])

def req1(digits):
    ''' Return boolean indicating if digits pass first requirement. '''
    # Return False if not contains subsequence in ['abc', 'def', ..., 'xyz']
    retval = False
    for i in range(len(digits) -2):
        if digits[i] + 1 == digits[i+1] and digits[i] + 2 == digits[i+2]:
            retval = True
            break
    return retval

def req2(digits):
    ''' Return boolean indicating if digits pass second requirement. '''
    # Return False if contains char in ['i','l]
    return False if 8 in digits or 11 in digits else True

def req3(digits):
    ''' Return boolean indicating if digits pass third requirement. '''
    # Return False if not contains two sets of unequal doubles (e.g. 'aa', 'bb')
    retval = False
    for i in range(len(digits)-3):              # find first set of doubles
        if digits[i] == digits[i+1]:
            for j in range(i+2,len(digits)-1):  # set all lower digits to 0
                if digits[j] != digits[i] and digits[j] == digits[j+1]:
                    retval = True
                    break
            break  # no need to continue checking regardless of retval
    return retval

def is_valid_pswd(digits):
    ''' Return boolean indicating if password meets all requirements.'''
    # Note req2 is faster, so order it first.  If req2 fails, it should
    # not compute the other two and just return False.
    return all([req2(digits), req1(digits), req3(digits)])

def increment_digits(digits, base):
    ''' Increment digits odometer style using base.  Return boolean
         indicating whether increment was successful or not.
        Note, this function will not wrap around to 0.
    '''
    retval = False
    for i in range(len(digits)):
        if digits[len(digits)-1-i] != base - 1:
            retval = True
            digits[len(digits)-1-i] += 1
            for j in range(len(digits)-i, len(digits)):
                digits[j] = 0
            break
    return retval

pdigs = get_digits(pswd)    # convert old password to digits
partA, partB = None, None   
while increment_digits(pdigs, len(alphaIndex)) and partB == None:
    if is_valid_pswd(pdigs):
        if partA == None:      partA = get_word(pdigs)
        elif partB == None:    partB = get_word(pdigs)

logging.info("11a. The next valid password after {} is: {}.".format(pswd, partA))
logging.info("11b. The next next valid password after {} is: {}.".format(pswd, partB))

2023-12-05 11:01:42 INFO: 11a. The next valid password after hxbxwxba is: hxbxxyzz.
2023-12-05 11:01:42 INFO: 11b. The next next valid password after hxbxwxba is: hxcaabcc.


# Day 12:  JSAbacusFramework.io

## 12a.  What is the sum of all numbers in the document?

## 12b.  What is the sum of all non-red numbers in the document?

In [15]:
filename = 'prob12input.txt'
lines = get_string_input(filename)
#lines = ['[1,2,3]','{"a":2,"b":4}','[[[3]]]','{"a":{"b":4},"c":-1}','{"a":[-1,1]}','[-1,{"a":1}]','[]','{}']
line = json.loads(lines[0])

def get_sum_numbers(obj, red=False):
    ''' Return the sum of all (valid) numbers contained in object. 
        All numbers are valid if red==False.  If red==True, then 
         numbers are valid if they are not a part of a dictionary
         containing a key or value of "red".
        This function uses recursion to do a depth first search
         and returns the sum of all valid numbers in obj.
    '''
    retval = 0
    if type(obj) == type(int()):
        retval = obj
    elif red == True and obj == 'red':
        retval = None
    elif type(obj) == type(list()):
        for v in obj:
            val1 =  get_sum_numbers(v, red)
            retval = retval if val1 == None else retval + val1
    elif type(obj) == type(dict()):
        for k, v in obj.items():
            val1, val2 = get_sum_numbers(k, red), get_sum_numbers(v, red)
            if red == True and (val1 == None or val2 == None):
                retval = 0
                break
            else:
                retval += val1 + val2
    return retval

partA = get_sum_numbers(line)
partB = get_sum_numbers(line, red=True)
logging.info('12a. Sum of all numbers: {}'.format(partA,))
logging.info('12b. Sum of all non-red numbers: {}'.format(partB,))

2023-12-05 11:01:42 INFO: Got 1 lines from prob12input.txt.
2023-12-05 11:01:42 INFO: 12a. Sum of all numbers: 191164
2023-12-05 11:01:42 INFO: 12b. Sum of all non-red numbers: 87842


# Day 13:  Knights of the Dinner Table

## 13a.  What is the happiness level of the optimal seating arrangement?

## 13b.  Including you, what is the happiness level of the optimal seating arrangement?

In [16]:
filename = 'prob13input.txt'
lines = get_string_input(filename)

# Parse input into graph G
G = dict()
line_re = re.compile('^(.*?) would ((gain)|(lose)) (\d+) happiness units by sitting next to (.*?)\.$') 
for l in lines:
    m1 = line_re.match(l)
    assert m1 != None
    u, sign, dist, v = m1.groups(1)[0], m1.groups(1)[1], int(m1.groups(1)[4]), m1.groups(1)[5]
    dist = -1*dist if sign == 'lose' else dist
    Nu, Nv = G.get(u, dict()), G.get(v, dict())  # get neighborhoods
    Nu[v] = Nu.get(v,0) + dist                   # add to neighborhoods
    Nv[u] = Nv.get(u,0) + dist                   # add to neighborhoods
    G[u], G[v] = Nu, Nv                          # update neighborhoods
# Finished parsing input into graph G

def extend_hamiltonian_path(dist, subpath, G, minmax='min'):
    ''' Return shortest (longest) hamiltonian path in G beginning with subpath.
        dist is an integer
        subpath is a list of vertex strings from G
        G is a 2 layer dictionary of distances; G[u][v] = distance from u to v
        minmax in ['min', 'max'] means shortest/longest path desired    
        Returns [dist, path] where dist is the distance of path and path
         begins with subpath.

        Note: This is a slightly modified function taken from question 9.
    '''
    assert 0 <= len(subpath) <= len(G)
    orderFunc = min if minmax == 'min' else max
    retval = None
    u = None if len(subpath) == 0 else subpath[-1]  # get last vertex in path
    if len(subpath) == len(G): # if all vertices on subpath, return to start
        retval = [ dist + G[u][subpath[0]], subpath.copy() + [subpath[0]] ]  
    # else add new vertext not on subpath
    for v in [w for w in G.keys() if w not in subpath]:
        newPath = [v] if u == None else subpath.copy() + [v]
        newDist = 0 if u == None else dist + G[u][v]  # dist = 0 for 1st vertex
        shortLong = [newDist, newPath]  # default if path complete
        if len(newPath) <= len(G):   # Is path unfinished?
            shortLong = extend_hamiltonian_path(newDist, newPath, G, minmax)
        if retval == None or orderFunc(retval[0],shortLong[0]) != retval[0]:
            retval = shortLong
    return retval

# part A
start = list(G.keys())[0]  # it doesn't matter where path starts
partA = extend_hamiltonian_path(0, [start], G, 'max')

# part B
G['me'] = dict()
for v in G.keys():   # Add 'me' to graph
    G['me'][v], G[v]['me'] = 0, 0
partB = extend_hamiltonian_path(0, [start], G, 'max')

logging.info("14a. Optimal seating has happiness {}".format(partA[0],))
logging.info("14b. Optimal seating that includes me has happiness {}".format(partB[0],))    

2023-12-05 11:01:42 INFO: Got 56 lines from prob13input.txt.
2023-12-05 11:01:43 INFO: 14a. Optimal seating has happiness 733
2023-12-05 11:01:43 INFO: 14b. Optimal seating that includes me has happiness 725


# Day 14:  Reindeer Olympics

## 14a.  How far is the furthest reindeer after 2503 seconds?

## 14b.  How many points does the winning reindeer have?

In [17]:
filename = 'prob14input.txt'
lines = get_string_input(filename)
    
# Parse input dict()
stats = dict()
line_re = re.compile('^(.*?) can fly (\d+) km/s for (\d+) seconds, but then must rest for (\d+) seconds\.$') 
for l in lines:
    m1 = line_re.match(l)
    assert m1 != None
    reindeer, rate, travelTime, restTime = m1.groups(1)[0:4]
    stats[reindeer] = {'rate':int(rate), 'travel':int(travelTime), 'rest':int(restTime)}

# part A
maxtime, maxdist, maxdeerA = 2503, 0, None
for r in stats.keys():       # for each reindeer r
    time, traveled = 0, 0 
    while time < maxtime:    # compute travel for r; cap at maxtime
        traveled += stats[r]['rate'] * min(stats[r]['travel'], maxtime - time)
        time = min(maxtime, time + stats[r]['travel'] + stats[r]['rest'])
    #logging.info("{} traveled {} km in {} seconds".format(r, traveled, time))
    if traveled > maxdist:   # keep track of deer that traveled the farthest
        maxdist, maxdeerA = traveled, r

# part B
dists = dict(zip(stats.keys(),[0 for _ in stats.keys()]))
points = dict(zip(stats.keys(),[0 for _ in stats.keys()]))
maxtime, maxdist, maxdeerB = 2503, 0, []
for time in range(maxtime):
    for r in stats.keys():  # Move each reindeer, keep track of maxdist and maxdeer
        if 0 < (time+1) % (stats[r]['travel'] + stats[r]['rest']) <= stats[r]['travel']:
            dists[r] += stats[r]['rate']
        if dists[r] > maxdist:          # Update maxdeer and maxdist if needed
            maxdist, maxdeerB = dists[r], [r]
        elif dists[r] == maxdist and r not in maxdeerB:
            maxdeerB.append(r)
    for r in maxdeerB:       # Award points
        points[r] += 1
maxPoints, maxPointDeer = max([[y for y in reversed(x)] for x in points.items()])

logging.info("14a. {} travels the farthest in {} seconds: {} km".format(maxdeerA, maxtime, maxdist))
logging.info("14b. {} earns the most points in {} seconds: {}".format(maxPointDeer, maxtime, maxPoints))

2023-12-05 11:01:43 INFO: Got 9 lines from prob14input.txt.
2023-12-05 11:01:43 INFO: 14a. Vixen travels the farthest in 2503 seconds: 2660 km
2023-12-05 11:01:43 INFO: 14b. Blitzen earns the most points in 2503 seconds: 1256


# Day 15:  Science for Hungry People

## 15a.  What is the maximum possible cookie score?

## 15b.  What is the maximum possible cookie score if the cookie has 500 calories?

In [18]:
filename = 'prob15input.txt'
lines = get_string_input(filename)

# Parse input dict()
stats = dict()
line_re = re.compile('^(.*?): capacity ([\d-]+), durability ([\d-]+), flavor ([\d-]+), texture ([\d-]+), calories ([\d-]+)\s*$') 
for l in lines:
    m1 = line_re.match(l)
    assert m1 != None
    ing, cap, dur, fla, tex, cal = m1.groups(1)[0:6]
    stats[ing] = {'cap':int(cap), 'dur':int(dur), 'fla':int(fla), 'tex':int(tex), 'cal':int(cal)}
# Finished parsing input

def get_splits(splits):
    ''' Return a list of three split locations '''
    retval = splits
    if len(splits) == 1:    # three splits in same loc
        retval = splits * 3
    elif len(splits) == 2:  # 2 splits in first loc, 1 split in second loc
        retval = sorted([splits[0], splits[0], splits[1]])
    return retval

def put_n_balls_in_four_bins(n, splits):
    ''' Return a 4-tuple that gives the number of balls in each of 4 bins. 
        splits is a list of integers giving location of splits
        Imagine a line of n items, the n+1 locations before, after, or in
         between those n items are the possible "split locations".  Once
         the three splits are inserted into the line, the number of balls
         in each bin is the number of items between splits.
    '''
    assert len(splits) == 3
    return [splits[0], splits[1]-splits[0], splits[2]-splits[1], n-splits[2]]

# Part A and Part B
maxScoreA, maxDistroA, maxScoreB, maxDistroB = 0, [], 0, []

# Reformat input
calorieScores = [stats[i]['cal'] for i in stats.keys()]
propScores = [[stats[i][p] for i in stats.keys()] for p in ['cap','dur','fla','tex']]

n = 100
for splits in tt.chain(tt.combinations(range(n+1),3),   # splits in three diff locs
                       tt.permutations(range(n+1),2),   # 2 splits in one loc, 1 elsewhere
                       tt.combinations(range(n+1),1)):  # 3 splits in the same loc
    distro = put_n_balls_in_four_bins(n, get_splits(splits))
    calories = dot_product(calorieScores, distro)
    ingrScores = [dot_product(prop, distro) for prop in propScores]
    cookieScore = product(ingrScores) if all([s > 0 for s in ingrScores]) else 0
    if cookieScore > maxScoreA:  #partA
        maxScoreA, maxDistroA = cookieScore, distro.copy()
    if calories == 500 and cookieScore > maxScoreB:  #partB
        maxScoreB, maxDistroB = cookieScore, distro.copy()

logging.info("15a. Best cookie score: {}".format(maxScoreA,))
logging.info("15b. Best cookie score having 500 calories: {}".format(maxScoreB,))

2023-12-05 11:01:43 INFO: Got 4 lines from prob15input.txt.
2023-12-05 11:01:43 INFO: 15a. Best cookie score: 13882464
2023-12-05 11:01:43 INFO: 15b. Best cookie score having 500 calories: 11171160


# Day 16:  Aunt Sue

## 16a.  What is the number of Sue that gave you the gift?

## 16b.  What is the (real) number of Sue that gave you the gift?

In [19]:
filename = 'prob16input.txt'
lines = get_string_input(filename)

# Parse input dict()
stats = dict()
line_re = re.compile('^Sue (\d+): (\S+?): (\d+), (\S+?): (\d+), (\S+?): (\d+)\s*$') 
for l in lines:
    m1 = line_re.match(l)
    assert m1 != None
    num, prop1, val1, prop2, val2, prop3, val3 = m1.groups(1)[0:7]
    stats[int(num)] = {prop1:int(val1), prop2:int(val2), prop3:int(val3)}
# Finished parsing input

message = {'children': 3,'cats': 7,'samoyeds': 2,'pomeranians': 3,'akitas': 0,
           'vizslas': 0,'goldfish': 5,'trees': 3,'cars': 2,'perfumes': 1}

partA = None
for num, sue in stats.items():
    causalSue = all([sue[prop] == message.get(prop,None) for prop in sue.keys()])
    if causalSue and partA == None:
        partA = num

partB = None
for num, sue in stats.items():
    sueProps = []
    for prop in sue.keys():
        if prop in ['cats','trees']:
            sueProps.append(sue[prop] > message.get(prop,None))
        elif prop in ['pomeranians','goldfish']:
            sueProps.append(sue[prop] < message.get(prop,None))
        else:
            sueProps.append(sue[prop] == message.get(prop,None))
    causalSue = all(sueProps)
    if causalSue and partB == None:
        partB = num

logging.info("16a. Gift was given by Aunt Sue {}".format(partA,))
logging.info("16b. Gift was given by (real) Aunt Sue {}".format(partB,))

2023-12-05 11:01:43 INFO: Got 500 lines from prob16input.txt.
2023-12-05 11:01:43 INFO: 16a. Gift was given by Aunt Sue 213
2023-12-05 11:01:43 INFO: 16b. Gift was given by (real) Aunt Sue 323


# Day 17:  No Such Thing as Too Much

## 17a.  How many different ways can you store the eggnog?

## 17b.  How many different ways can you store the eggnog using the minimum number of containers?


In [20]:
filename = 'prob17input.txt'
containers = sorted(get_integer_input(filename), reverse=True)

# Part A and Part B
numDist = dict()
for numContainers in range(1, len(containers)):
    numDist[numContainers] = 0
    for subsetContainers in tt.combinations(containers, numContainers):
        if sum(subsetContainers) == 150:
            numDist[numContainers] += 1
partA = sum(numDist.values())
partB = min([k for k, v in numDist.items() if v > 0])

logging.info("17a. Number of container configurations: {}".format(partA,))
logging.info("17b. Minimum number of containers and number of configurations: {}".format(partB,))

2023-12-05 11:01:43 INFO: Got 20 integers from prob17input.txt.
2023-12-05 11:01:44 INFO: 17a. Number of container configurations: 1304
2023-12-05 11:01:44 INFO: 17b. Minimum number of containers and number of configurations: 4


# Day 18:  Like a GIF For Your Yard

## 18a.  How many lights are on after 100 steps?

## 18b.  With alterations, how many lights are on after 100 steps?

In [21]:
filename = 'prob18input.txt'
lines = get_string_input(filename)

#lines = ['.#.#.#','...##.','#....#','..#...','#.#..#','####..']

def get_neighbor_vals(r, c, n, diagonal=True):
    ''' Return the values of all neighbors in grid. 
        r, c, n are integer row, column, and dimension of nxm matrix, resp
        diagonal is boolean indicating if diagonals are nieghbors
    '''
    retval = []
    ind = get_grid_index(r, c, n, n)
    for neighbor in get_grid_neighborhood(ind, n, diagonal):
        neighbor_r, neighbor_c = neighbor//n, neighbor%n
        retval.append(M[neighbor_r][neighbor_c])
    return retval
    
def step_grid(M, cornersOn=False):
    ''' Return a new M with same dimensions that has been stepped as in 
         problem description.
        cornersOn: boolean (for part B) indicating if corners are always on
    '''
    retval = []
    n = len(M)
    for r in range(n):  # Create replacement rows, one at a time
        newRow = []
        for c in range(n):
            newVal = '.'  # default 'off' until contradicted
            neighborsOn = get_neighbor_vals(r, c, n, True).count('#')
            if cornersOn and r in [0,n-1] and c in [0,n-1]:
                newVal = '#'
            elif neighborsOn == 3 or neighborsOn == 2 and M[r][c] == '#':
                newVal = '#'
            newRow.append(newVal)  # add value to row
        retval.append(newRow)  # add row to new matrix
    return retval

# Part A and B
numSteps = 100
for cornersOn in [False, True]:
    M = [[ch for ch in line] for line in lines]
    if cornersOn:  # Turn corners on if needed
        M[0][0], M[0][-1], M[-1][0], M[-1][-1] = '#', '#', '#', '#'
    # Run algorithm
    for step in range(numSteps):
        M = step_grid(M, cornersOn)
    # Save output
    if cornersOn:    partB = sum([row.count('#') for row in M]) 
    else:            partA = sum([row.count('#') for row in M])

logging.info('18a. Lights on after {} steps: {}'.format(numSteps, partA))
logging.info('18b. With corners always on, lights on after {} steps: {}'.format(numSteps, partB))

2023-12-05 11:01:44 INFO: Got 100 lines from prob18input.txt.
2023-12-05 11:01:50 INFO: 18a. Lights on after 100 steps: 821
2023-12-05 11:01:50 INFO: 18b. With corners always on, lights on after 100 steps: 886


# Day 19:  Medicine for Rudolph

## 19a.  How many distinct new molecules can be created from the input?

## 19b.  What is the minimum number of steps required to create the input string?

In [22]:
filename = 'prob19input.txt'
lines = get_string_input(filename)

#lines = ['e => H','e => O','H => HO','H => OH','O => HH','','HOHOHO']

# Parse input dict()
repls = dict()
molecule = None
repl_re = re.compile('^([A-Za-z]+) => ([A-Za-z]+)\s*$')
start_re = re.compile('^([A-Za-z]+)\s*$')
for l in lines:
    m1, m2 = repl_re.match(l), start_re.match(l)
    assert l == '' or m1 != None or m2 != None
    if m1 != None:
        preRepl, postRepl = m1.groups(1)[0:2]
        vals = repls.get(preRepl,[])
        vals.append(postRepl)
        repls[preRepl] = vals
    elif m2 != None:
        molecule = m2.groups(1)[0]
    elif l != '':
        logging.info("Error: '{}'".format(l,))

#logging.info("repls = {}".format(repls,))
#logging.info("molecule = {}".format(molecule,))

partA = set()
#molecules.add(molecule)
for index in range(len(molecule)):
    #singles
    for post in repls.get(molecule[index],[]):
        newMolecule = molecule[:index] + post + molecule[index+1:]
        partA.add(newMolecule)
    #doubles
    if index < len(molecule) - 1:
        for post in repls.get(molecule[index:index+2],[]):
            newMolecule = molecule[:index] + post + molecule[index+2:]
            partA.add(newMolecule)

logging.info("19a. Number of new molecules: {}".format(len(partA),))

## For Part B, note that there are essentially four types of transformations:

# 1) x => xx           (adds one molecule)
# 2) x => xRnxAr       (adds three molecules)
# 3) x => xRnxYxAr     (adds five molecules)
# 4) x => xRnxYxYxAr   (adds seven molecules)

# where x is not in ['Rn','Y','Ar']

# This means the Rn and Ar molecules come in pairs.  So each such pair will
# will be a part of a three molecules expansion.  Each Y present will be add
# 2 additional molecules to that expansion.

# Thus the total number of steps is equal to 
# (Number of Molecules - 1) - 2*(Number of Rn-Ar pairs) - 2*(Number of Ys)

totalMols = sum([1 for ch in molecule if 'A' <= ch <= 'Z'])
pairsRnAr = molecule.count('Rn')
numYs = molecule.count('Y')
partB = totalMols - 2*pairsRnAr - 2*numYs - 1
logging.info("19b. The minimal number of steps to create the molecule: {}".format(partB,))

# I attempted a brute force of all possible reductions, starting from the molecule
# and moving toward e.  This is not possible (in a reasonable amount of time) due
# to the enormity of the number of paths from the molecule to e.  I attempted a couple
# optimizations by reducing "intenal" values such as the x's in any of the items in
# W = {ZRnxAr, ZRnxYxAr, ZRnxYxYxAr} where X is any valid string and Z is a string
# with nothing from {Rn, Y, Ar}.  Each X can be reduced to single value.  But
# eventually a string ww...wZ (with w \in W) is encountered consisting of 12
# blocks from W.  This is way too long to churn through all possible reductions.
# 
# After being disappointed with the complexity I searched online and it seems that
# the approach I implemented above is the desired approach.  The intention of the
# puzzle maker was to give something that did not yield to brute force, no matter
# how clever the approach (at least in any reasonable amount of time).  There are
# many that claim to have produced a brute force solution, but they are, in my
# opinion, not "real" solutions and only work because of the mechanics of the
# particular grammar.  Most "solutions" employ a greedy approach (or a random approach)
# to achieve a single solution.  Since the number of steps for each solution is always
# the same (see analysis above) the found solution is correct, however, this is just 
# "a solution" and in all such examples no effort is made to ensure it is 
# "the minimal solution".  The only way to prove a solution is the minimal solution
# is through careful analysis of the grammar.
#
# This problem is entirely disappointing.

2023-12-05 11:01:50 INFO: Got 45 lines from prob19input.txt.
2023-12-05 11:01:50 INFO: 19a. Number of new molecules: 535
2023-12-05 11:01:50 INFO: 19b. The minimal number of steps to create the molecule: 212


# 20.  Infinite Elves and Infinite Houses

## 20a.  What is the first house to receive at least 29000000 presents?

## 20b.  After modification, what is the first house to receive at least 29000000 presents?

In [23]:
divisors, primes, target = {}, [], 29000000
partA, partB = None, None

# iterate through integers, note primes and sets of divisors for each
for n in range(2, 2*target):
    bound, nDivs = math.sqrt(n), None
    for p in primes:      # loop through known primes looking for divisor
        if p > bound: break  # if hasn't happened yet, it ain't happening
        if n % p == 0:             # check if p divides n
            nDivs = divisors[n//p]              # each divisor of n
            others = {p*k for k in nDivs}       #  must be in nDivs
            divisors[n] = nDivs.union(others)   #  or in others
            break     # only need one nontrivial divisor to find them all
    if nDivs == None:         # this means n is prime
        primes.append(n)      #  add to primes
        divisors[n] = {1, n}  #  and set its divisors

    # Check for solutions for each integer
    if partA == None and 10*sum(divisors[n]) > target:
        partA = n
    if partB == None and 11*sum([d for d in divisors[n] if 50*d >= n]) > target:
        partB = n
    
    # If all solutions found, bail
    if all([partA, partB]):
        break

logging.info("20a. The lowest house is {}.".format(partA,))
logging.info("20b. The lowest house is {}.".format(partB,))

2023-12-05 11:01:55 INFO: 20a. The lowest house is 665280.
2023-12-05 11:01:55 INFO: 20b. The lowest house is 705600.


# 21.  RPG Simulator 20XX

## 21a.  What is the least amount of gold you can spend and still win?

## 21b.  What is the most amount of gold you can spend and still lose?

In [24]:
weapons = {'Dagger':{'cost':8,'damage':4,'armor':0},
           'Shortsword':{'cost':10,'damage':5,'armor':0},
           'Warhammer':{'cost':25,'damage':6,'armor':0},
           'Longsword':{'cost':40,'damage':7,'armor':0},
           'Greataxe':{'cost':74,'damage':8,'armor':0}}
armor = {'Leather':{'cost':13,'damage':0,'armor':1},
         'Chainmail':{'cost':31,'damage':0,'armor':2},
         'Splintmail':{'cost':53,'damage':0,'armor':3},
         'Bandedmail':{'cost':75,'damage':0,'armor':4},
         'Platemail':{'cost':102,'damage':0,'armor':5},
         'Nothing':{'cost':0,'damage':0,'armor':0}}
rings = {'Damage +1':{'cost':25,'damage':1,'armor':0},
         'Damage +2':{'cost':50,'damage':2,'armor':0},
         'Damage +3':{'cost':100,'damage':3,'armor':0},
         'Defense +1':{'cost':20,'damage':0,'armor':1},
         'Defense +2':{'cost':40,'damage':0,'armor':2},
         'Defense +3':{'cost':80,'damage':0,'armor':3},
         'Nothing1':{'cost':0,'damage':0,'armor':0},
         'Nothing2':{'cost':0,'damage':0,'armor':0}}
boss = {'Hit Points': 103,'damage': 9,'armor': 2}
hero = {'Hit Points': 100,'cost':0,'damage': 0,'armor': 0}

def fight(h, b):
    ''' Return the winner of the fight given the stats for
         the hero (h) and the boss (b)
    '''
    retval = 'boss'
    bossDamagePerRound = max(1, b['damage']-h['armor'])
    heroDamagePerRound = max(1, h['damage']-b['armor'])
    bossRoundsToWin = math.ceil(hero['Hit Points'] / bossDamagePerRound)
    heroRoundsToWin = math.ceil(boss['Hit Points'] / heroDamagePerRound)
    if heroRoundsToWin <= bossRoundsToWin:
        retval = 'hero'
    return retval

def equip_hero(hero, gear):
    ''' Copy hero, add property values and return. '''
    retval = hero.copy()
    for stat in gear:
        for property in ['cost','damage','armor']:
            retval[property] += stat[property]
    return retval

# Part A and Part B
# Small space, just exhaust and separate into heroWins and bossWins.
heroWins, bossWins = [], []
for wStats, aStats in tt.product(weapons.values(), armor.values()):
    for r1, r2 in tt.combinations(rings.items(), 2):
        if r2[0] == 'Nothing1':   # Avoid a little duplication
            continue              #  since 'Nothing1' == 'Nothing2'
        heroCopy = equip_hero(hero, [wStats, aStats, r1[1], r2[1]])
        if fight(heroCopy, boss) == 'hero':
            heroWins.append(heroCopy)
        else:
            bossWins.append(heroCopy)

partA = sorted(heroWins, key=lambda x: x['cost'])[0]['cost']
partB = sorted(bossWins, key=lambda x: x['cost'], reverse=True)[0]['cost']

logging.info("21a. The minimal amount to spend and still win: {}".format(partA,))
logging.info("21a. The maximal amount to spend and still lose: {}".format(partB,))

2023-12-05 11:01:55 INFO: 21a. The minimal amount to spend and still win: 121
2023-12-05 11:01:55 INFO: 21a. The maximal amount to spend and still lose: 201


# 22.  Wizard Simulator 20XX

## 22a.  What is the least amount of mana you can spend and still win?

## 22b.  With a harder scenario, what is the least amount of mana you can spend and still win?


In [25]:
boss = {'Hit Points': 58,'damage':9}
hero = {'Hit Points': 50,'mana':500,'armor':0}

spells = {'Magic Missile':{'cost':53,'damage':4},
          'Shield':{'cost':113,'armor':7,'rounds':6},
          'Recharge':{'cost':229,'rounds':5},
          'Poison':{'cost':173,'rounds':6},
          'Drain':{'cost':73,'damage':2,'heal':2}}

class State():
    ''' Holds information needed that describes the state of and
         path of a particular game.
    '''
    def __init__(self):
        self.boss = None
        self.hero = None
        self.turns = []
        self.activePlayer = 'hero'
        self.spent = 0
        self.rounds = {'Shield':0, 'Poison':0, 'Recharge':0}
        self.winner = None
        self.value = None

    def copy(self):
        ''' Return a new State that is a copy of self. '''
        retval = State()
        retval.boss = self.boss.copy()
        retval.hero = self.hero.copy()
        retval.turns = self.turns.copy()
        retval.activePlayer = self.activePlayer
        retval.spent = self.spent
        retval.rounds = self.rounds.copy()
        retval.winner = self.winner
        return retval

    def is_state_better_non_strict(self, state):
        ''' Return Ture if self > state (for some definition of '>')
            Note, returning False does not mean self better than state.
        '''
        retval = False
        if all([self.spent >= state.spent,
                self.hero['mana'] <= state.hero['mana'],
                self.activePlayer == state.activePlayer,
                self.winner == state.winner,
                self.boss['Hit Points'] >= state.boss['Hit Points'],
                self.hero['Hit Points'] <= state.hero['Hit Points'],
                self.rounds['Shield'] <= state.rounds['Shield'],
                self.rounds['Poison'] <= state.rounds['Poison'],
                self.rounds['Recharge'] <= state.rounds['Recharge']]):
            retval = True
        return retval

    def is_state_better_strict(self, state):
        ''' Return True if self > state (for some definition of '>')
            Note, returning False does not mean self better than state. 
        '''
        retval = False
        if all([self.spent >= state.spent,
                self.hero['mana'] == state.hero['mana'],
                self.activePlayer == state.activePlayer,
                self.winner == state.winner,
                self.boss['Hit Points'] == state.boss['Hit Points'],
                self.hero['Hit Points'] == state.hero['Hit Points'],
                self.rounds['Shield'] == state.rounds['Shield'],
                self.rounds['Poison'] == state.rounds['Poison'],
                self.rounds['Recharge'] == state.rounds['Recharge']]):
            retval = True
        return retval

    def check_for_win(self):
        ''' Update self.winner if appropriate. '''
        assert self.winner == None
        if self.boss['Hit Points'] <= 0:
            self.winner = 'hero'
        elif self.hero['Hit Points'] <= 0:
            self.winner = 'boss'

    def round_effects(self, penalty):
        ''' Update state by applying round effects and checking for winner. '''
        assert self.winner == None
        self.rounds['Shield'] = max(0, self.rounds['Shield']-1)
        if penalty:
            self.hero['Hit Points'] -= 1
            self.check_for_win()
        if self.winner == None:
            if self.rounds['Shield'] == 0:
                self.hero['armor'] = 0
            if self.rounds['Poison'] > 0:
                self.rounds['Poison'] -= 1
                self.boss['Hit Points'] -= 3
                self.check_for_win() 
            if self.rounds['Recharge'] > 0:
                self.rounds['Recharge'] -= 1
                self.hero['mana'] += 101

    def take_boss_turn(self):
        ''' Extend self by taking a boss turn.  This assumes round effects
             have already been added and produced no winner.
        '''
        assert self.winner == None and self.activePlayer == 'boss'
        self.hero['Hit Points'] -= max(1, (self.boss['damage'] - self.hero['armor']))
        self.activePlayer = 'hero'
        self.turns.append('b')
        self.check_for_win()

    def take_hero_turn(self):
        ''' Extend self by taking a hero turn.  This assumes round effects
             have already been added and produced no winner.
            Each of five moves is considered and each possible move generates
             a new state which is returned.  If no new states are generated,
             then hero has lost the game and current state is updated and returned.        
        '''
        retval = []
        assert self.winner == None and self.activePlayer == 'hero'
        for spell in spells.keys():   # try to cast each spell
            if self.hero['mana'] >= spells[spell]['cost'] and self.rounds.get(spell,0) == 0:
                nS = self.copy()                 # can cast spell, create new state
                nS.turns.append(spell[0])        #  and apply affects of spell to state
                nS.hero['mana'] -= spells[spell]['cost']   
                nS.spent += spells[spell]['cost']
                if spell == 'Magic Missile':
                    nS.boss['Hit Points'] -= spells[spell]['damage']
                elif spell == 'Drain':
                    nS.boss['Hit Points'] -= spells[spell]['damage']
                    nS.hero['Hit Points'] = min(50, nS.hero['Hit Points']+spells[spell]['heal'])
                else:
                    nS.rounds[spell] = spells[spell]['rounds']
                    if spell == 'Shield':
                        nS.hero['armor'] = spells[spell]['armor']
                nS.activePlayer = 'boss'
                nS.check_for_win()
                retval.append(nS)
        if len(retval) == 0:   # means no spell cast, wizard is a loser
            self.winner = 'boss'
            self.turns.append('mana')
            retval.append(self)
        return retval
    
    def output(self, turnNum=None):
        ''' Helper output function that is useful for debugging. '''
        logging.info("{} turns = {}".format(turnNum,self.turns))
        logging.info("{} boss = {}".format(turnNum,self.boss))
        logging.info("{} hero = {}".format(turnNum,self.hero))
        logging.info("{} rounds = {}".format(turnNum,self.rounds))
        logging.info("{} activePlayer = {}".format(turnNum,self.activePlayer))
        logging.info("{} winner = {}".format(turnNum,self.winner))
        logging.info("{} spent = {}".format(turnNum,self.spent))
# Finished with State() definition.

def take_single_turn(state, penalty):
    ''' Return the new states found from single turn (boss or hero)
        Round effects are first applied and if winner is found by
        then the state is returned without completing round
    '''
    retval = []
    assert state.winner == None
    state.round_effects(penalty)
    if state.winner != None:
        state.turns.append('r')  # r means round effects
        retval = [state]
    elif state.activePlayer == 'boss':
        state.take_boss_turn()
        retval = [state]
    elif state.activePlayer == 'hero':
        retval = state.take_hero_turn()
    return retval

def insertions_sort_of_states(newStates, orderedQueue):
    ''' Return the orderedQueue with newStates inserted.  Returned
         queue should be in increasing order of state.spent
        Discard any newState if unneeded, that is...
         - if boss wins newState
         - if state is found in queue that is "better"
    '''
    retval = orderedQueue
    start_index = 0
    for newS in sorted(newStates, key=lambda x: x.spent):
        toAddAtEnd = True  # Add at end if internal insert fails
        if newS.winner == 'boss':
            continue  # Do not clutter queue with garbage
        for i in range(start_index, len(retval)):
            if newS.is_state_better_non_strict(retval[i]):
                # queue state is "better than" newS; discard newS
                toAddAtEnd = False
            elif newS.spent < retval[i].spent:  # enter newS here
                retval = retval[:i] + [newS] + retval[i:]
                toAddAtEnd = False
            if not toAddAtEnd:
                start_index = i  # start next search at start_index
                break
        if toAddAtEnd:   # if True, state to be added at end of queue
            retval.append(newS)       # add state and ensure remaining
            start_index = len(retval) #  states added to end as well
    return retval

def find_min_mana_spend_state(boss, hero, penalty=False):
    ''' Return a State that achieves the minimum number
         of mana spent.  The algorithm employed is essentially
         a modified Dijkstra's algorithm.  The minimum state
         in the queue is extended and each new state is added
         to the queue if a "better" state is not found first.
    '''
    retval = None
    # Create initial state and add to queue
    begin = State()
    begin.boss, begin.hero = boss.copy(), hero.copy()
    queue = [begin]
    # While queue is nonempty, get state and extend into new states
    while len(queue) > 0:
        s = queue.pop(0)  # get state with lowest spent value
        if s.winner == 'hero':
            retval = s
            break
        # Get new states from state s
        newStates = take_single_turn(s, penalty)
        # Add new states to queue; maintain sorted order of queue
        queue = insertions_sort_of_states(newStates, queue)
    return retval

# Part A and Part B
winnerA = find_min_mana_spend_state(boss, hero)
winnerB = find_min_mana_spend_state(boss, hero, penalty=True)

logging.info("22a. The minimum mana spent that still wins: {}".format(winnerA.spent,))
logging.info("22b. In the harder scenario, the minimum mana spent that still wins: {}".format(winnerB.spent,))

# Instead of an insertion sort for the queue above, a tree queue or heap queue
# may have been more efficient.

2023-12-05 11:01:56 INFO: 22a. The minimum mana spent that still wins: 1269
2023-12-05 11:01:56 INFO: 22b. In the harder scenario, the minimum mana spent that still wins: 1309


# 23.  Opening the Turing Lock

## 23a.  What is the value of register b when the program finishes?

## 23b.  With a slight modification, what is the value of register b when the program finishes?

In [26]:
filename = 'prob23input.txt'
lines = get_string_input(filename)

# Parsing input into commands
line_re = re.compile('^(\S+) (.*?)\s*$')
commands = []
for l in lines:
    m1 = line_re.match(l)
    assert m1 != None
    command, arguments = m1.groups(1)[0], m1.groups(1)[1].split(', ')
    if command in ['inc','tpl','hlf']:
        assert len(arguments) == 1
        commands.append([command, arguments[0]])
    elif command == 'jmp':
        assert len(arguments) == 1
        commands.append([command, int(arguments[0])])
    elif command in ['jie','jio']:
        assert len(arguments) == 2
        commands.append([command, [arguments[0], int(arguments[1])]])
# Finished parsing input into commands

def run_regsiter(regs, commands, maxSteps=None):
    ''' Run the program as defined in problem and return regs when
         finished.  While index is still referencing a valid command,
         execute it.  
    '''
    step, index = 0, 0
    while index < len(commands):
        step += 1
        com, argmts = commands[index]
        index += 1  # adding 1 by default
        if com == 'inc':    regs[ argmts ] += 1
        elif com == 'tpl':  regs[ argmts ] *= 3
        elif com == 'hlf':  regs[ argmts ] >>= 1 # aka divide by 2
        elif com == 'jmp':  index += argmts - 1  # already added 1
        elif com == 'jie':
            if regs[ argmts[0] ] % 2 == 0:
                index += argmts[1] - 1           # already added 1
        elif com == 'jio':
            if regs[ argmts[0] ] == 1:
                index += argmts[1] - 1           # already added 1
        if maxSteps != None and step > maxSteps: # throttle if needed
            break
    return regs

# Part A
regsA = run_regsiter({'a':0, 'b':0}, commands)
# Part B
regsB = run_regsiter({'a':1, 'b':0}, commands)
assert regsA['a'] == 1 and regsB['a'] == 1  # indicates successful finish

logging.info("23a. The value in register b when finished is: {}".format(regsA['b'],))
logging.info("23b. The value in register b when finished is: {}".format(regsB['b'],))

2023-12-05 11:01:56 INFO: Got 48 lines from prob23input.txt.
2023-12-05 11:01:56 INFO: 23a. The value in register b when finished is: 255
2023-12-05 11:01:56 INFO: 23b. The value in register b when finished is: 334


# 24.  It Hangs in the Balance

## 24a.  What is the quantum entanglement for the ideal configuration?

## 24b.  With four groups, what is the quantum entaglement for the ideal configuration?

In [27]:
filename = 'prob24input.txt'
weights = sorted(get_integer_input(filename), reverse=True)

# I'm using the fact that all values are distinct and odd.  Using
# these facts makes the algorithm faster.  The algorithm would
# need adjustment if either fact was not true.
def partition_values(values, n, target):
    ''' Return a list of subsets from value where the sum of
         each subset equals target.  Each subset should have
         size at least equal to n.
    '''
    retval = []
    assert sum(values) % target == 0
    numSubsetsRemaining = int(sum(values) / target)
    maxSize = int(len(values) / numSubsetsRemaining)  
    assert n <= maxSize
    for subsetSize in range(n,maxSize+1):
        if subsetSize % 2 != target % 2:  # all weights are odd, so parity
            continue           # of subsetSize and target must be the same
        for subset in tt.combinations(values, subsetSize):
            if sum(subset) == target:
                # One subset found; recursively partition remaining elements
                remaining = [x for x in values if x not in subset]
                if len(remaining) > 0:
                    retval = partition_values(remaining, subsetSize, target)
                    if len(retval) > 0:
                        retval = [subset] + retval
                        break
                else:
                    retval = [subset]
                    break
        if len(retval) > 0:
            break
    return retval

def find_min_quantum_entanglement(weights, numGroups):
    ''' Return an integer that is the minimum quantum entanglement.
        Attempt to split weights into numGroups groups where each
         group has the same sum (target).  Among all possible solutions
         where the number of elements in the first group is minimal,
         return the minimal product of the elements in the first group.
    '''
    retval = None
    assert sum(weights) % numGroups == 0
    prods = []   # Hold products of g1Vals for valid partition of weights
    target = int(sum(weights)/numGroups)
    maxSize = math.floor(len(weights)/numGroups)
    logging.info("numGroups = {}, target = {}, maxSize = {}".format(numGroups, target, maxSize))  

    for g1Size in range(1, maxSize+1):
        if g1Size % 2 != target % 2:  # Shortcut: all weights are odd, so parity
            continue                  # of g1Size and target must be the same

        for g1Vals in tt.combinations(weights, g1Size):
            if sum(g1Vals) == target:
                # Found subset that works, try to partition rest of values
                remaining = [x for x in weights if x not in g1Vals]
                check = partition_values(remaining, g1Size, target) 
                if len(check) > 0:
                    # Success!  check = [g2Vals, g3Vals, ...]
                    prods.append(product(g1Vals))
                    # don't break, all possible g1Vals needed

        if len(prods) > 0:
            retval = min(prods)  # Solutions found for g1Size, get min
            break                #  and return
    return retval

partA = find_min_quantum_entanglement(weights, 3)
partB = find_min_quantum_entanglement(weights, 4)

logging.info("24a. The minimum quantum entanglement for three groups: {}".format(partA,))
logging.info("24b. The minimum quantum entanglement for four groups: {}".format(partB,))

2023-12-05 11:01:56 INFO: Got 28 integers from prob24input.txt.
2023-12-05 11:01:56 INFO: numGroups = 3, target = 508, maxSize = 9
2023-12-05 11:01:57 INFO: numGroups = 4, target = 381, maxSize = 7
2023-12-05 11:01:58 INFO: 24a. The minimum quantum entanglement for three groups: 10439961859
2023-12-05 11:01:58 INFO: 24b. The minimum quantum entanglement for four groups: 72050269


# 25.  Let It Snow

## 25a.  What is the code at row 2978, column 3083?

## 25b.  Free Star!

In [28]:
targetRow, targetCol = 2978, 3083

def get_triangle_number(n):
    assert n >= 0
    return int(n*(n+1)/2)

def get_index_in_grid(row, col):
    assert row > 0 and col > 0
    return get_triangle_number(row+col-2) + col

def euclidean_algorithm(a, b):
    ''' Return the gcd(a,b) '''
    retval = b
    k = a % b
    if k > 0:
        retval = euclidean_algorithm(b, k)
    return retval

codeNum = get_index_in_grid(targetRow, targetCol)
logging.info("codeNum = {}".format(codeNum,))

a, g, m = 20151125, 252533, 33554393   # m is prime
phi = m-1      # = 2*2*2*29*61*2371

# The following isn't required, but I found it informative.
logging.info("gcd({}, {}) = {}".format(m, g, euclidean_algorithm(m, g)))

# By group theory, the order of g in the multiplicative group of Z_m
# must divide phi.  There are more clever ways of finding that order
# using phi, but it doesn't take that long to find it directly.
# But the order of g is (m-1)/2... so finding it directly doesn't
# really help at all, but I'll leave the code here anyway for fun.

x = 1
for order in range(1, phi+10000):
    x = (x * g) % m
    if x == 1:
        break
assert phi % order == 0  # Just checking
logging.info("The order of {} in multiplicative group Z_{} is {}".format(g, m, order))

# Then g^codeNum % m = g^(codeNum%order) % m because
#  g^codeNum % m = g^((q*order) + r) % m  where q = codeNum//order and r = codeNum%order
#                = (g^order)^q(g^r) % m
#                = (1)^q(g^r) % m
#                = g^r % m

# 1 value is a
# 2 value is a*g   % m
# 3 value is a*g^2 % m
# ...
# n value is a*g^{n-1} % m
newCodeNum = codeNum % order
partA = a
for i in range(newCodeNum-1):
    partA = (partA * g) % m
logging.info("25a. The value of at ({}, {}) is {}".format(targetRow,targetCol,partA))

2023-12-05 11:01:58 INFO: codeNum = 18361853
2023-12-05 11:01:58 INFO: gcd(33554393, 252533) = 1
2023-12-05 11:02:00 INFO: The order of 252533 in multiplicative group Z_33554393 is 16777196
2023-12-05 11:02:00 INFO: 25a. The value of at (2978, 3083) is 2650453
